In [1]:
import pandas as pd
df = pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()


,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


## Word Embedding

In [2]:
# Tokenize
def tokenize(text):
  text = text.lower() #convert all to lowercase
  text = text.replace('?','') #replace ? with ''
  text = text.replace("'","") # replace ' with ''
  return text.split() #split words

In [3]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [4]:
#Vocabulary
vocab = {'<UNK>':0} #intialize dictionary with UNK for unknown values

In [12]:
def build_vocab(row):
  print(row['question'], row['answer'])
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])
  print(tokenized_question, tokenized_answer)

  merged_tokens = tokenized_question + tokenized_answer
  print(merged_tokens)

  for token in merged_tokens:

    if token not in vocab: #add token to vocab
      vocab[token] = len(vocab) #index = current vocab length


In [13]:
df.apply(build_vocab, axis=1)


What is the capital of France? Paris
['what', 'is', 'the', 'capital', 'of', 'france'] ['paris']
['what', 'is', 'the', 'capital', 'of', 'france', 'paris']
What is the capital of Germany? Berlin
['what', 'is', 'the', 'capital', 'of', 'germany'] ['berlin']
['what', 'is', 'the', 'capital', 'of', 'germany', 'berlin']
Who wrote 'To Kill a Mockingbird'? Harper-Lee
['who', 'wrote', 'to', 'kill', 'a', 'mockingbird'] ['harper-lee']
['who', 'wrote', 'to', 'kill', 'a', 'mockingbird', 'harper-lee']
What is the largest planet in our solar system? Jupiter
['what', 'is', 'the', 'largest', 'planet', 'in', 'our', 'solar', 'system'] ['jupiter']
['what', 'is', 'the', 'largest', 'planet', 'in', 'our', 'solar', 'system', 'jupiter']
What is the boiling point of water in Celsius? 100
['what', 'is', 'the', 'boiling', 'point', 'of', 'water', 'in', 'celsius'] ['100']
['what', 'is', 'the', 'boiling', 'point', 'of', 'water', 'in', 'celsius', '100']
Who painted the Mona Lisa? Leonardo-da-Vinci
['who', 'painted', 't

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [14]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 'paris': 7,
 'germany': 8,
 'berlin': 9,
 'who': 10,
 'wrote': 11,
 'to': 12,
 'kill': 13,
 'a': 14,
 'mockingbird': 15,
 'harper-lee': 16,
 'largest': 17,
 'planet': 18,
 'in': 19,
 'our': 20,
 'solar': 21,
 'system': 22,
 'jupiter': 23,
 'boiling': 24,
 'point': 25,
 'water': 26,
 'celsius': 27,
 '100': 28,
 'painted': 29,
 'mona': 30,
 'lisa': 31,
 'leonardo-da-vinci': 32,
 'square': 33,
 'root': 34,
 '64': 35,
 '8': 36,
 'chemical': 37,
 'symbol': 38,
 'for': 39,
 'gold': 40,
 'au': 41,
 'which': 42,
 'year': 43,
 'did': 44,
 'world': 45,
 'war': 46,
 'ii': 47,
 'end': 48,
 '1945': 49,
 'longest': 50,
 'river': 51,
 'nile': 52,
 'japan': 53,
 'tokyo': 54,
 'developed': 55,
 'theory': 56,
 'relativity': 57,
 'albert-einstein': 58,
 'freezing': 59,
 'fahrenheit': 60,
 '32': 61,
 'known': 62,
 'as': 63,
 'red': 64,
 'mars': 65,
 'author': 66,
 '1984': 67,
 'george-orwell': 68,
 'currency': 69,
 'unit

In [17]:
len(vocab)

324

In [19]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token]) #add index of word to list
    else:
      indexed_text.append(vocab['<UNK>'])#add index of UNK to list

  return indexed_text

In [20]:
text_to_indices("What is RNN", vocab)

[1, 2, 0]

## DataLoader

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader

In [22]:
class QADataset(Dataset):

  def __init__(self, df, vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    #convert current question to indices
    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    #convert current ans to indices
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)

    return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [23]:
dataset = QADataset(df, vocab)

In [24]:
dataset[0]

(tensor([1, 2, 3, 4, 5, 6]), tensor([7]))

In [25]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [26]:
for question, answer in dataloader:
  print(question, answer[0])

tensor([[ 78,  79, 288,  81,  19,  14, 289]]) tensor([85])
tensor([[ 1,  2,  3, 69,  5,  3, 70, 71]]) tensor([72])
tensor([[ 10, 140,   3, 141, 270,  93, 271,   5,   3, 272]]) tensor([273])
tensor([[ 10,  29, 130, 131]]) tensor([132])
tensor([[ 42,  86,  87, 241, 242,  19,  39, 243]]) tensor([244])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([128])
tensor([[  1,   2,   3, 221,   5, 222, 223, 224]]) tensor([225])
tensor([[  1,   2,   3,  33,  34,   5, 245]]) tensor([246])
tensor([[  1,   2,   3,  69,   5, 155]]) tensor([156])
tensor([[1, 2, 3, 4, 5, 6]]) tensor([7])
tensor([[  1,   2,   3,   4,   5, 109]]) tensor([317])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([136])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([316])
tensor([[10, 29,  3, 30, 31]]) tensor([32])
tensor([[  1,   2,   3,   4,   5, 113]]) tensor([114])
tensor([[ 42, 107,   2, 108,  19, 109]]) tensor([110])
tensor([[1, 2, 3, 4, 5, 8]]) tensor([9])
tensor([[ 10, 140,   3, 141, 142,  12, 

## RNN

In [27]:
import torch.nn as nn

In [28]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50) #embedding layer
    self.rnn = nn.RNN(50, 64, batch_first=True) #rnn layer
    self.fc = nn.Linear(64, vocab_size) #output layer

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [29]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [30]:
learning_rate = 0.001
epochs = 20

In [31]:
model = SimpleRNN(len(vocab))

In [32]:
criterion = nn.CrossEntropyLoss() #loss
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [33]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 523.887549
Epoch: 2, Loss: 457.408464
Epoch: 3, Loss: 379.782628
Epoch: 4, Loss: 317.742112
Epoch: 5, Loss: 265.195534
Epoch: 6, Loss: 215.979610
Epoch: 7, Loss: 170.514786
Epoch: 8, Loss: 131.659542
Epoch: 9, Loss: 100.339111
Epoch: 10, Loss: 76.087261
Epoch: 11, Loss: 58.247924
Epoch: 12, Loss: 45.582162
Epoch: 13, Loss: 36.129793
Epoch: 14, Loss: 29.431363
Epoch: 15, Loss: 24.149083
Epoch: 16, Loss: 20.168503
Epoch: 17, Loss: 17.118085
Epoch: 18, Loss: 14.521469
Epoch: 19, Loss: 12.593628
Epoch: 20, Loss: 10.909156


In [47]:
def predict(model, question, threshold=0.5):

  # convert question to numbers
  numerical_question = text_to_indices(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)
  print(f"max-probability: {value}, Index: {index}\n")

  if value < threshold:
    print("This topic isn't in my current knowledge base.")
    return

  print(list(vocab.keys())[index]) #return value in vocab

In [48]:
predict(model, "What is the largest planet in our solar system?")

max-probability: tensor([0.8910], grad_fn=<MaxBackward0>), Index: tensor([23])

jupiter


In [49]:
predict(model, "What is global warming?")

max-probability: tensor([0.0690], grad_fn=<MaxBackward0>), Index: tensor([102])

This topic isn't in my current knowledge base.
